In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv
import os
import random
import string
import seaborn as sns
import math
import re
import warnings
import lightgbm as lgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce

from datetime import datetime
from dateutil.parser import parse
from sklearn import metrics
from unidecode import unidecode
from sklearn.metrics import roc_auc_score

from itertools import combinations

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
plt.style.use('ggplot')

%matplotlib inline

In [2]:
data_version_input = "01-10-2020"

In [3]:
data_version = "11-10-2020-v4"

In [4]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

In [5]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)

In [6]:

f =  "currentLocationCity"
vc1 = sorted(df_train[f].value_counts().index)
print(vc1)
print(len(vc1))
print("######################################################################################################")
vc2 = sorted(df_test[f].value_counts().index)
print(vc2)
print(len(vc2))
a = []
for i in vc2:
    if i not in vc1:
        a.append(i)
print("######################################################################################################")
print(a)


print(len(a))



['??oan Hùng', 'A Luoi (1)', 'Abu Dhabi', 'Ajaccio', 'American Falls', 'Amherst', 'Amphoe Song Phi Nong', 'An Biên', 'An Dương District', 'An Giang (4)', 'An Khê', 'An Lac', 'An Lão', 'An Lê', 'An Nhon', 'An Ninh', 'An Phong', 'An Phu', 'An Thi', 'An Thuan', 'Angiang', 'Anh Son', 'Anini-y', 'Ap An Thai (1)', 'Ap Bao Binh', 'Ap Ben Cam', 'Ap Ben Cau', 'Ap Binh Chanh (1)', 'Ap Binh Duong (1)', 'Ap Binh Hung', 'Ap Binh Quang', 'Ap Binh Thuan', 'Ap Hung Yen', 'Ap Lam Phu', 'Ap Loi Nhon', 'Ap My Tho', 'Ap Nhon My', 'Ap Tan Ha', 'Ap Tan Phu', 'Ap Tan Thanh', 'Ap Thanh Dong', 'Ap Thanh Nhut', 'Ap Ã?ao', 'Australind', 'Aylmer', "B'Sar Dong", "B'sre", 'Ba Don', 'Ba Hòn', 'Ba Ria', 'Ba Thuoc', 'Ba Tri', 'Ba Vì', 'Bac Binh', 'Bac Can', 'Bac Giang', 'Bac Ha', 'Bac Kan', 'Bac Lieu', 'Bac Ninh', 'Bac Quang', 'Bac Son', 'Bach Cau', 'Bach Ma', 'Bai Thuong', 'Bali', 'Ban Daknong', 'Ban Gioc', 'Ban Krong Buk', 'Ban Krong Pack', "Ban M'drack", 'Ban Me Thuot', 'Ban Nong Bua Noi', 'Bandar-e-Emam Khomeyni',

In [7]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G", "Field_3", "Field_38"],
    
    "cat_str" : ["Field_4", "Field_12","Field_36", "Field_54", "Field_62","Field_65","Field_66",\
                 "info_social_sex", "brief", "gioiTinh","currentLocationState", "Field_61", \
                 "currentLocationCountry", 'homeTownState', 'data.basic_info.locale',\
                "homeTownCountry", "Field_55", "currentLocationCity"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh", "Field_34"],
    
    "address" : ["Field_48", "Field_49", "Field_56", 'homeTownName',\
                 "homeTownCity", "diaChi", "currentLocationName"],
    
    "drop" : ["Field_18", "Field_45", "Field_68", "Field_46"]
}

consider transform: "brief" \
consider drop: "Field_56", "maCv"


In [8]:
df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["address"], axis = 1)

In [9]:
def fix38(x):
    if x == 0.0 or x == "0" or x == "0.0":
        return 0
    elif x == 1.0 or x == "1":
        return 1
    return np.nan

In [10]:
def fix54(x):
    if x == "1" or x == "01" or x == ' P. Tân Phú':
        return "VN"
    elif x in ['IT', 'GB', 'SE', 'TK', 'BD']:
        return np.nan
    return x

In [11]:
def fix55(x):
    if x in ['BC', 'BG', 'CN', 'CO', 'DB', 'EL', 'FO', 'H5', 'KX', 'N7', 'NR', 'SD', 'SJ', 'ST', 'WD']:
        return np.nan
    return x

In [12]:
def fix61(x):
    if x == 'Chất độc hóa học':
        return np.nan
    return x

In [13]:
def fixmaCv(x):
    lower = str(x).lower()
    if any([x in lower for x in ["nhân viên", "kế toán", "thủ kho", "tạp vụ",\
                                "thu ngân", "phóng viên", "giao dịch", "operator", "cnv",\
                                "huấn luyện viên", "ktoán", "nv", "lễ tân", "thư ký ",\
                                "gsbh", "tiếp viên", "nhõn viờn", "chăm sóc", "hướng dẫn viên", \
                                "kiểm phẩm",  "kĩ sư",  "cấp dưỡng", "kiểm gấp",\
                                "kiểm soát viên", "kiểm ngân", "nv", "lưu hóa", "nhân  viên", "lưu trữ viên", \
                                "phát hành báo", "ktv", "kiến trúc sư", "kinh tế viên", "staff", "checker",\
                                "sỹ quan", "dịch vụ", "kiểm tra viên", \
                                "cộng tác viên", "qc ksqt", "nhân vên", "kế hoạch sản xuất", "nhaõn vieõn", \
                                "thí nghiệm", "thông kê", "kiểm hàng", "k? s?", "thư ký", "hợp đồng", \
                                "thẩm phán", "thao tác viên", "thủ quỹ", "lấy mẫu", "LÁI XE".lower(), \
                                "biên tập", "ksxd", "phiên dịch", "tb tuyên huấn", "sales", "thường trực", \
                                "trợ lý", "tiếp thực", "giỏm sỏt", \
                                "trình dược viên", "btđtn", "phúng viờn", \
                                "thông dịch", "tiếp thị", "tư vấn", "tổng đài", "vi tính", "viên chức", \
                                "văn phòng", "văn thư", 'Thiết Kế'.lower(), "vận hành", "nhan vien", \
                                "kiểm tra chất lượng", "pbt", "phi công", "laptop",\
                                "phụ trách kiểm hàng", "quay phim", "thư viện viên", \
                                "trực buồng", "techcombank", "cảng vụ viên", "trainee", "bhxh", "marketing", \
                                "tuyên truyền viên",  "kế hoạch", "ks", \
                                "an toàn viên", "CHỈ SỐ VIÊN".lower(), \
                                "thu mua", "giám éịnh viên", "kiểm định viên",  "giám định", \
                                "quản trị mạng", "thơợ", "thống kê", "thủ  quỹ",  \
                                "điện thoại", "điện thọai", "đội thông tin", "đón  tiếp khách hàng", \
                                "đơn hàng", "đại diện", "điều phối viên", "n.v", "bán vé", "c«ng nh©n", \
                                "gs camera",  "máy dán", "tv",  "đồ họa"]]):
        return "NV"
    elif any([x in lower for x in ["kỹ","chuyên viên","kĩ thuật viên","kĩ thuật", "kỹ thuật viên",\
                                 "kỷ thuật viên","lập trình viên", "chuyờn viờn", "chuyªn viªn",]]):
        return "KTV"
    elif any([x in lower for x in ["cán bộ", "cán sự", "bí thư đoàn", "cb", "phường đội", "trưởng công an", \
                                  "công an viên", "an ninh viên","tổ viên", "công chức", "quản sinh", \
                                   "bí thư xã  đoàn","điều tra viên", "đăng kiểm", "cỏn bộ","đtv", "trực ban",\
                                  ]]):
        return "CB"
    elif any([x in lower for x in ["tổ trưởng", "giám sát", "quản lý", "cửa hàng trưởng", "chuyền trưởng", "kho",\
                                  "phó", "pp", "nhóm trưởng", "trưởng phòng", "trưởng ca", "trưởng tàu khách", \
                                  "trưởng nhóm", "cửa hàng trưởng", "trưởng bộ", "tổng phụ trách", "đào tạo", \
                                  "phụ trách phòng", "quản doanh", "vhtt", "trưởng ban", "dự án", "nhân sự",\
                                  "chỉ huy", "triển khai", "trưởng kiểm soát", "điều hành", "đội trưởng", \
                                  "chỉ huy", "qlsx", "t? tr??ng", "may"]]):
        return "QL"
    elif any([x in lower for x in ["công",  "bảo vệ", "thợ", "lao động", "bán hàng",\
                                "phụ kho", "phụ việc", "phết", "bảo mẫu", "phục vụ", "đóng gói", "ép cao tần", \
                                "cn", "sản Phẩm", "giao nhận","cụng nhõn", "gia công", "kiÓm tra".lower(), \
                                "cô nuôi", "bảo dưỡng", "c.n", "coi bồn", "cạo mủ", "coõng nhaõn", "khai thác",\
                                "cod", "bốc xếp", "bếp", "lái máy", "lđpt", "khuôn", "hàng rong", "phụ bếp",\
                                "phụ tá", "thủ công", "dệt", "sửa chữa", "băng chuyền", "lắp ráp", \
                                "chà nhám", "cn", "bếp phụ", "chụp hình", "KHUNG SắT".lower(), "kcs chuyền", \
                                "lao động", "thành hình", "lao công", "cônh nhân", "thi công", "cán lần 1", \
                                "lái phụ máy", "thủy thủ", "gác ghi", "quay ghi", "bảo trì", "bảo mẩu", \
                                "máy 3 tàu", "giao hàng", "trực chính", "hỗ trợ", "lái xúc gạt", "công  nhân",\
                                "lđhđ", "lao d?ng", "c. nhân", "pv nước", "hđlđ", "vệ sinh", "phụ", "vệ sỹ", \
                                "xếp hộp", "barista", "hàn điện", "thuỷ thủ", "xây dựng", "b?o v?", \
                                "mài phá", "xếp da", "worker", "cong nhan", "pha chế", \
                                "máy trưởng", "phụ trách sản xuất", "trải vải", "YếN TIệC".lower(), "gấp xếp", \
                                "in sơ đồ", "lđ phổ thông", "mài", "nhồi mút", "ép", "gói", "đóng", \
                                "đội viên", "đánh máy", "đứng máy", "ủi", "săm", 'atlđ']]):
        return "CN"
    elif any([x in lower for x in ["lái xe", "tài xế", "phụ xe","lái cẩu", "vân hành", ]]):
        return "TX"
    elif any([x in lower for x in ["giáo viên", "gv", "giảng viên", "chủ nhiệm lớp", "giáo viến", \
                                   "Giáo viến".lower(), "mầm non"]]):
        return "GV"
    elif any([x in lower for x in ["kinh doanh", "kinh  doanh"]]):
        return "KD"

    elif any([x in lower for x in ["hiệu trưởng", "chủ tịch", "pgđ", "giám đốc"]]):
        return "GĐ"
    elif any([x in lower for x in ["điều dưỡng", "hộ lý", "nữ hộ sinh", "h? lý", "hộ sinh"]]):
        return "ĐD"
    elif any([x in lower for x in ["y sĩ", "dược sỹ", "bác sĩ", "bác sỹ", "dược", "dược tá", "thú y", "y sỹ",\
                                  "y tế", "y công", "y tá", "dựơc tá", "ytth"]]):
        return "BS"
    elif any([x in lower for x in ["cử nhân", "cđ", "học viên", "tc", "cao đẳng", "cao đẵng", "trung cấp"]]):
        return "HS"
    elif any([x in lower for x in ["diễn viên", "ca sĩ"]]):
        return "DV-CS"
    elif any([x in lower for x in ["undefined", "nguyễn thế hưng", "ln091", "thieu chuc danh", "th noọng hẹt", \
                                  "875 lk", "-", "p.xnk", "cỏn b?", '22568', "unt", "01019882", "xtnt", "ltv", \
                                  "plđg hc", "mia", "pqtd", "kttntl", "llqs", "kcsmhm", "nt", "dtm", "i", "qc",\
                                  '/  /', '1', '14', 'a', 'chủ hộ', 'coõng vuù', 'kcs', 'pk b2c',\
                                   'xt2', 'kcs', 'sxct']]):
        return np.nan
    return x

In [14]:
def combine_gender(s):
    x, y = s
    
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1)
    return df

In [15]:
def calculate_age(born):
    born = datetime.strptime(born,'%Y-%m-%d').date()
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [16]:
def fixhomeTownState(x):
    if x in ['Hunan', 'Kachin State', 'Distrito Federal', 'Bizerte Governorate', 'Central Java',\
             'Svay Rieng Province', 'Distrito Nacional', 'County Mayo', 'Odisha', 'Lazio', 'Nevada',\
             'Daegu', 'Arizona', 'Equatoria', 'Fukuoka Prefecture', 'Taoyuan City', 'Kandal Province', \
             'State of Mexico', 'Hebei', 'New South Wales', 'Capital Governorate', 'Oregon']:
        return np.nan
    return x

In [17]:
def fixhomeTownCountry(x):
    if x in ['Australia', 'Bahrain', 'Dominican Republic', 'Ireland', 'South Sudan', 'Tunisia']:
        return np.nan
    return x

In [18]:
def fixcurrentLocationState(x):
    if x in ['Ilhas', 'Daegu', 'Auckland Region', 'Basse-Normandie', 'Chongqing', "Provence-Alpes-Côte d'Azur",\
             'Rajshahi Division', 'Hubei', 'Mandalay', 'Missouri', 'Pathum Thani', 'Central Java', 'New Hampshire',\
             'Odisha', 'South Australia', 'Chon Buri', 'Haute-Normandie', 'Kirov Oblast', 'Primorsky Krai', 'Arizona',\
             'Ulsan', 'Kanagawa Prefecture', 'Metro Manila', 'Kachin State', 'Svay Rieng Province', 'Nagano Prefecture',\
             'Shiga Prefecture', 'Oslo', 'Indiana', 'Hunan', 'Bolikhamsai Province', 'Krasnoyarsk Krai',\
             'Distrito Nacional', 'Chiba Prefecture']:
        return np.nan
    return x

In [19]:
def fixdata_basic_info_locale(x):
    if x == "_VN":
        return 'vi_VN'
    elif x in ['lv_LV', 'ru_RU']:
        return np.nan
    return x

In [20]:
def fixcurrentLocationCountry(x):
    if x in ['Macau', 'New Zealand', 'Dominican Republic']:
        return np.nan
    return x

In [21]:
def fixcurrentLocationCity(x):
    y = x
    lower = str(x).lower()
    if any([x in lower for x in ["buon ma thuot", "ban me thuot"]]):
        return "Buon Me Thuot"
    elif any([x in lower for x in ["buôn hô"]]):
        return "Buôn Hô"
    elif any([x in lower for x in ["bình thanh"]]):
        return "Binh Thanh"
    elif any([x in lower for x in ["bu dang"]]):
        return "Bu Dang"
    elif any([x in lower for x in ["angiang", "an giang"]]):
        return "An Giang"
    elif any([x in lower for x in ["cam ranh"]]):
        return "Cam Ranh"
    elif any([x in lower for x in ["fan thiet", "phan thiet"]]):
        return "Phan Thiet"
    elif any([x in lower for x in ["haiphong"]]):
        return "Hai Phong"
    elif any([x in lower for x in ["ha tin"]]):
        return "Ha Tinh"
    elif any([x in lower for x in ["chau thanh"]]):
        return "Chau Thanh"
    elif any([x in lower for x in ["thanh tri"]]):
        return "Thanh Tri"
    elif any([x in lower for x in ["hòa an"]]):
        return "Hoa An"
    elif any([x in lower for x in ["hòa bình"]]):
        return "Hoa Binh"
    elif any([x in lower for x in ["kiên an"]]):
        return "Kien An"
    elif any([x in lower for x in ["long thành"]]):
        return "Long Thanh"
    elif any([x in lower for x in ["nhơn trạch"]]):
        return "Nhon Trach"
    elif any([x in lower for x in ["ninbinh", "ninh bình"]]):
        return "Ninh Binh"
    elif any([x in lower for x in ["ninh hòa"]]):
        return "Ninh Hoa"
    elif any([x in lower for x in ["phan ranh", "phan rang"]]):
        return "Phan Rang"
    elif any([x in lower for x in ['thái hõa', 'thái hòa']]):
        return "Thai Hoa"
    elif any([x in lower for x in ['plây cu', 'plây ku', "pleiku"]]):
        return "Pleiku"
    elif any([x in lower for x in ['quan nam']]):
        return "Quang Nam"
    elif any([x in lower for x in ['thanh bình', "thanh binh"]]):
        return "Thanh Binh"
    elif any([x in lower for x in ["vietri"]]):
        return "Viet Tri"
    elif any([x in lower for x in ["vinh loi"]]):
        return "Vinh Loi"
    elif any([x in lower for x in ["vinh loc", "vĩnh lộc"]]):
        return "Vinh Loc"
    elif any([x in lower for x in ["vĩnh phúc", "vinh phuc"]]):
        return "Vinh Phuc"
    elif any([x in lower for x in ["vinh hoa"]]):
        return "Vinh Hoa"
    elif any([x in lower for x in ["vung tau"]]):
        return "Vung tau"
    elif any([x in lower for x in ["bình thanh"]]):
        return "Binh Thanh"
    elif any([x in lower for x in ["hung loc"]]):
        return "hung loc"
    elif any([x in lower for x in ["đà lạt"]]):
        return "Da Lat"
    elif any([x in lower for x in ["my th"]]):
        return "My Tho"
    elif any([x in lower for x in ["gia loc"]]):
        return "Gia Loc"
    elif any([x in lower for x in ["an phu", "an phú"]]):
        return "Gia Loc"
    elif any([x in lower for x in ["lac lam"]]):
        return "Lac Lam"
    elif any([x in lower for x in ["bach lieu"]]):
        return "Bac Lieu"
    elif any([x in lower for x in ["bình gia", "binh gia"]]):
        return "Binh Gia"
    elif any([x in lower for x in ["cam pha"]]):
        return "Cam Pha"
    elif any([x in lower for x in ["cau ngang"]]):
        return "Cau Ngang"
    elif any([x in lower for x in ["chau phú"]]):
        return "Chau Phu"
    elif any([x in lower for x in ["dubai"]]):
        return "Dubai"
    elif any([x in lower for x in ["linh xuân", "linh xúân"]]):
        return "Linh Xuan"
    elif any([x in lower for x in ['Phu My'.lower()]]):
        return 'Phu My'
    elif any([x in lower for x in ["phu giáo"]]):
        return 'Phu Giao'
    elif any([x in lower for x in ["phù ninh"]]):
        return 'Phu Ninh'
    elif any([x in lower for x in ["thânh liêm"]]):
        return 'Thanh Liêm'
    elif any([x in lower for x in ["tan thanh", "tân thành"]]):
        return 'Tan Thanh'
    elif any([x in lower for x in ["gia kiêm"]]):
        return 'Gia Kiem'
    elif any([x in lower for x in ["yên bài"]]):
        return 'Yên Bái'
    elif any([x in lower for x in ["tân phú"]]):
        return 'Tan Phu'
    elif any([x in lower for x in ["bình chánh", "binh chanh"]]):
        return 'Binh Chanh'
    elif any([x in lower for x in ["binh duong"]]):
        return 'Binh Duong'
    elif any([x in lower for x in ["hung yen"]]):
        return 'Hung Yen'
    elif any([x in lower for x in ["bac can"]]):
        return 'Bac Kan'
    elif any([x in lower for x in ["bình long"]]):
        return 'Binh Long'
    elif any([x in lower for x in ["songkau"]]):
        return 'Song Cau'
    elif any([x in lower for x in ["hai-doung", "hai dzung"]]):
        return 'Hai Duong'
    elif any([y in ['??ó Nhât', 'Amphoe Muang Chon Buri', 'An Hoa', 'Angola', 'Ap Binh Ta (2)',\
                                   'Ap Chanh (4)', 'Ap Go Dau', 'Ap Phu Ly (2)', 'Ap Suoi Cat', 'Ap Tan Hoa',\
                                   'Asaka-shi', 'Auckland', 'Australia Plains', 'Ban Ea Phe', 'Ban Nam Ca Dinh',\
                                   'Bavet', 'Bhubaneswar', 'Bo Trach', 'Bon Yok Djou', 'Boston', 'Bích Trì', \
                                   'Caen', 'Cam Thuy', 'Can Gio', 'Can Loc', 'Can Thanh', 'Carson City', \
                                   'Catumbella', 'Chaozhou', 'Cho Lach', 'Cua Tung', 'Daegu', 'Dalton', \
                                   'Do Hai', 'Don Dang', 'Dong Huong', 'Duclair', 'Fukaya-shi', 'Funabashi-shi', \
                                   'Giang Thon', 'Grayson', 'Ha Vi', 'Haidöng', 'Hau Loc', 'Himeji-shi', \
                                   'Hoa Dong', 'Hoa Loc', 'Houston', 'Hung Long', 'Huong Tra', 'Janpan', 'Kirow', \
                                   'Komaki-shi', 'Korea', 'Lak', 'Lang Que Nho', 'Liangping', 'Liên Chiểu', \
                                   'Loc Binh', 'Local', 'Luc Yen', 'Macau', 'Madagui', 'Manila', 'Marseille', \
                                   'Mau Lam', 'My Hoi', 'Mâ\x80?kar Da Te', 'Na Hang', 'Nagano-shi', 'Nerima-ku', \
                                   'Nganan', 'Nghia Hung', 'Nha Long', 'Nhân Son (1)', 'Omihachiman-shi', 'Oslo', \
                                   'Pac Nam', 'Pathum Thani', 'Pha Le', 'Pho Ben Sung', 'Phoenix', 'Phu Hoa', \
                                   'Phu Huu', 'Phu Thu', 'Phuoc Lai', 'Phú Phong', 'Phú Túc', 'Phường Tân Tạo A', \
                                   'Pingtang', 'Quan Ba', 'Quỳnh Lôi', 'Santo Domingo', 'Tan Binh', 'Tan Hiep', \
                                   'Tay Dang', 'Teurlang Dong', 'Thach Thanh', 'Thanh Khê', 'Thanh-Nhan', \
                                   'Thien Giao', 'Thieu Hoa', 'Thuy Khue', 'Thác Bà', 'Tràm Chim', 'Tu Ly', \
                                   'Tu Ã?oan', 'Tân Quang', 'Tân Thới Hiệp', 'Ulsan', 'Van Hoi', 'Vinh Tu', \
                                   'Vladivostok', 'Vu Quang', 'Vung Liem', 'Vân Canh (1)', 'Wuhan', 'Xuan Dinh', \
                                   'Xuân Canh', 'Xóm Chu Chi', 'Yokohama', 'Ã?ong Trieu', 'Úc Lý']]):
        return np.nan
    return x

In [22]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj

In [23]:
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        df[col] = df[col].apply(date_normalize)

    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%m-%Y')
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        df[f'{cat}_startDate'] = pd.to_datetime(df[f"{cat}_startDate"], infer_datetime_format=True)
        df[f'{cat}_endDate'] = pd.to_datetime(df[f"{cat}_endDate"], infer_datetime_format=True)
        
        df[f'{cat}_startDate'] = df[f'{cat}_startDate'].dt.strftime('%m-%Y')
        df[f'{cat}_endDate'] = df[f'{cat}_endDate'].dt.strftime('%m-%Y')
        
        cat_cols.append(f'{cat}_startDate')
        cat_cols.append(f'{cat}_endDate')
    
    for col in cat_cols:
        df[col] = df[col]
        
    return df

In [24]:
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize)
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize)

    return df

In [25]:
df_transform["Field_38"] = df_transform["Field_38"].apply(fix38)
df_transform["Field_54"] = df_transform["Field_54"].apply(fix54)
df_transform["Field_55"] = df_transform["Field_55"].apply(fix55)
df_transform["Field_61"] = df_transform["Field_61"].apply(fix61)

df_transform["currentLocationState"] = df_transform["currentLocationState"].apply(fixcurrentLocationState)
df_transform["currentLocationCountry"] = df_transform["currentLocationCountry"].apply(fixcurrentLocationCountry)
df_transform["homeTownState"] = df_transform["homeTownState"].apply(fixhomeTownState)
df_transform["homeTownCountry"] = df_transform["homeTownCountry"].apply(fixhomeTownCountry)
df_transform['data.basic_info.locale'] = df_transform['data.basic_info.locale'].apply(fixdata_basic_info_locale)
df_transform["age"] = df_transform["ngaySinh"].map(lambda x: calculate_age(x) if x is not np.nan else x)
df_transform["maCv"] = df_transform["maCv"].apply(fixmaCv)
df_transform["null_sum"] = df_transform.isnull().sum(axis=1)
df_transform["currentLocationCity"] = df_transform["currentLocationCity"].apply(fixcurrentLocationCity)

df_transform = process_gender(df_transform)
df_transform = process_datetime_cols(df_transform)

df_transform = process_location(df_transform)

In [26]:
drop = ["gioiTinh", "info_social_sex", "ngaySinh"]
df_transform = df_transform.drop(drop, axis = 1)

In [27]:
categoricals = []
for header in df_transform.columns:
    if str(df_transform[header].dtypes) is "object":
        categoricals.append(header)
for col in categoricals:
    df_transform[col] = df_transform[col].astype(str)
lbl = LabelEncoder()
for col in categoricals:
    
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    
    df_transform[col] = df_transform[col].fillna('None')
    # Only take the common values in Train/Test-set
    common_vals = list(set(df_transform[col]))
    
    # Take if vals appeared both 5 times
    if col != "currentLocationCity":
        common_vals = set(df_transform[col].value_counts()[df_transform[col].value_counts()> 100].index)
    else:
        common_vals = set(df_transform[col].value_counts()[df_transform[col].value_counts()> 20].index)
    # Replace not-common values with "Missing" or np.NaN      
    df_transform.loc[~df_transform[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(df_transform[col].tolist())
    df_transform[col] = lbl.transform(df_transform[col])


In [28]:
fill_1 = []
for header in df_transform.columns:
    vc = df_transform[header].value_counts()
    if len(vc) < 2:
        fill_1.append(header)
for col in fill_1:
    df_transform[col] = df_transform[col].fillna(-1)

In [29]:
df_transform = df_transform.fillna(df_transform.mean())

In [30]:
import re
df_transform = df_transform.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [31]:
train_dev = pd.concat([df_train.id, df_train.label, df_transform.iloc[:len(df_train),:]], axis = 1)
test = pd.concat([df_test.id, df_transform.iloc[len(df_train):,:]], axis = 1)

In [32]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': 1,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.7,
    'min_data_in_leaf': 200,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'min_hessian': 0.01,
    'feature_fraction_seed': 2,
    'bagging_seed': 3
}

In [33]:
def compute_gini(labels, preds):
    fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    gini = 2 * auc - 1
    return gini

In [34]:
iter_predicttion = []
ginis = []
for i in range(4):
    pred_stack = []
    kf = KFold(n_splits = 8, shuffle=True)
    fold = kf.split(train_dev)
    for train_index, dev_index in fold:
        train1 = train_dev.iloc[train_index,2:]
        dev1 = train_dev.iloc[dev_index,2:]
        d_train = lgb.Dataset(train1, label = train_dev.loc[train_index]["label"])
        clf = lgb.train(params,d_train, 2500)
        predictions_test = clf.predict(test.iloc[:,1:])
        predictions_dev = clf.predict(dev1)
        pred_stack.append(predictions_test)
        gini = compute_gini(train_dev.loc[dev_index]["label"], predictions_dev)
        print(gini)
        ginis.append(gini)
    predictions = np.asarray(pred_stack)
    predictions = np.mean(predictions, axis = 0)
    iter_predicttion.append(predictions)

0.4882165564705683
0.48842201140945685
0.5011439885016016
0.4751693236748433
0.48550139504970957
0.4814309790102649
0.4747758224043257
0.4847714203915072
0.48779458083481186
0.4965428604223334
0.47427001026918036
0.47744787681004563
0.49718506631126336
0.4680607980146205
0.5026414345312133
0.4664919942389729
0.4882622100955558
0.4802946400585102
0.47181998437781636
0.4587363695078286
0.5022018899493603
0.4947365450817358
0.48362660685793424
0.49095331456503555
0.47028496254154595
0.49266456396011127
0.5065489047119054
0.48480358873762897
0.4760865953800377
0.470595820472111
0.49286201307082456
0.4771496379874922


In [35]:
print(np.mean(np.array(ginis)))

0.48410918017812976


In [36]:
predictions = np.asarray(iter_predicttion)
predictions = np.mean(predictions, axis = 0)
predictions = list(predictions)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
new_label = pd.DataFrame(predictions, columns = ["label"])

new_data_id = []
for i in range(len(new_label)):
    idx = len(df_train) + i
    new_data_id.append(idx)
new_data_id = pd.DataFrame(new_data_id, columns = ["id"])

new_data =  pd.concat([new_data_id, new_label, test.iloc[:,1:]], axis = 1)

In [37]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)
new_data.to_csv(f"../../data/kalapa/{data_version}/new_data.csv", index = False)